In [1]:
import torch
import torch.nn as nn


## Välkomna till denna MYCKET viktiga gruppuppgift!
 
Er utmaning är att gå igenom följande CNN-class nedan, och sedan förstå varenda steg i forward()-metoden.
 
Specifikt vill jag att ni förstår input- och output size för varje steg. Till er hjälp har ni redan svaren på input- och output för varje steg, men ni ska förstår *varför* det blir som det blir.
 
Ni har också tillgång till PDF:er som vi gått igenom tillsammans. Gå igenom tillsammans, om så behövs!
 
Diskutera i grupp steg-för-steg! Anteckna/rita om ni behöver för att förklara för era kamrater.
 
**bonus 1** om ni hinner bli klara: Hur många parametrar har varje layer? Hur många parametrar har nätverket totalt?
Kan ni komma fram till en formel för antalet parametrar per lager?
 
**bonus 2**: Leta runt på nätet efter en grym visualisering av filter-aktiveringar. Visa upp för klassen!
 
 
**Förutsättningar**
 
Anta att vi jobbar med ett dataset med RGB-bilder av storleken 28x28x3, och att vi försöker lösa ett multiclass-problem med 10 möjliga klasser.

---

CNNModel nedan.

In [21]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding='same')
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding='same')
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding='same')
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))

        self.flat = nn.Flatten()

        # Adjust the input size for fc4 based on the output of the final pooling layer
        self.fc4 = nn.Linear(3 * 3 * 64, 512)
        self.fc5 = nn.Linear(512, 10)

    def forward(self, x):                # PARAMETERS
        # Input = 28 x 28 x 3            # CNN
        x = self.relu(self.conv1(x))     # in_channels * out_channels * kernel_width * kernel_height + bias
        # Output = 28 x 28 x 32          #     3       *     32     *     3      *     3             +  32  = 864 + 32 = 896 parameters

        # Input = 28 x 28 x 32           # POOLING
        x = self.pool(x)                 # ((input_size_w - kernel_size_w + 2 * padding) / stride) + 1 = output_size_w
        # Output = 14 x 14 x 32          # (28 - 2 + 2 * 0) / 2 + 1 = 14

        # Input = 14 x 14 x 32
        x = self.relu(self.conv2(x))     # in_channels * out_channels * kernel_width * kernel_height + bias
        # Output = 14 x 14 x 64          #     32      *     64     *     3      *     3             +  64  = 18432 + 64 = 18 496 parameters

        # Input = 14 x 14 x 64
        x = self.pool(x)
        # Output = 7 x 7 x 64

        # Input = 7 x 7 x 64
        x = self.relu(self.conv3(x))     # in_channels * out_channels * kernel_width * kernel_height + bias
        # Output = 7 x 7 x 64            #     64      *     64     *      3      *         3        +  64  = 36 864 + 64 = 36 928 parameters
                                     
        # Input = 7 x 7 x 64
        x = self.pool(x)                 # ((input_size_w - kernel_size_w + 2 * padding) / stride) + 1 = output_size_w
        # Output = 3 x 3 x 64                   7        -      2        + 2 *    0     /    2     + 1 =       3

        # NOTERA OVAN ATT POOL HOPPAR ÖVER ETT PAR PIXLAR, OM INTE DET ÄR JÄMNT ANTAL
      
        # Flatten the tensor
        # Input = 3 x 3 x 64
        x = self.flat(x)
        # Output = 576 (3*3*64)

        # Input = 576
        x = self.relu(self.fc4(x))        # in_channels * out_channels + bias
        # Output = 512                    #     576     *    512       + 512 = 295 424 parameters

        # Input = 512
        x = self.fc5(x)              
        # Output = 10

        return x


## Answer to bonus 1
**Parameters per convolutional layer**

out_channels * (in_channels * kernel_width * kernel_height + 1) 

**Parameters per fully connected layer**

out_channels * (in_channels + 1)


In [22]:
model = CNNModel()
print(model)

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc4): Linear(in_features=576, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=10, bias=True)
)


In [23]:
from torchinfo import summary
summary(model, (1, 3, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
CNNModel                                 [1, 10]                   --
├─Conv2d: 1-1                            [1, 32, 28, 28]           896
├─ReLU: 1-2                              [1, 32, 28, 28]           --
├─MaxPool2d: 1-3                         [1, 32, 14, 14]           --
├─Conv2d: 1-4                            [1, 64, 14, 14]           18,496
├─ReLU: 1-5                              [1, 64, 14, 14]           --
├─MaxPool2d: 1-6                         [1, 64, 7, 7]             --
├─Conv2d: 1-7                            [1, 64, 7, 7]             36,928
├─ReLU: 1-8                              [1, 64, 7, 7]             --
├─MaxPool2d: 1-9                         [1, 64, 3, 3]             --
├─Flatten: 1-10                          [1, 576]                  --
├─Linear: 1-11                           [1, 512]                  295,424
├─ReLU: 1-12                             [1, 512]                  --
├